In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction import FeatureHasher
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# !pip install pyarrow

## Q1 - 19 columns

In [3]:
df_feb = pd.read_parquet('yellow_tripdata_2022-02.parquet')

In [4]:
df_feb

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.00,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.00,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.50,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.00,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.50,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979426,2,2022-02-28 23:50:00,2022-03-01 00:06:00,NaN,3.40,NaN,None,163,193,0,14.37,0.00,0.5,0.00,0.00,0.3,17.67,NaN,NaN
2979427,2,2022-02-28 23:06:57,2022-02-28 23:19:12,NaN,3.48,NaN,None,141,4,0,14.51,0.00,0.5,2.00,0.00,0.3,19.81,NaN,NaN
2979428,2,2022-02-28 23:48:13,2022-03-01 00:03:33,NaN,3.05,NaN,None,161,151,0,14.38,0.00,0.5,3.81,0.00,0.3,21.49,NaN,NaN
2979429,2,2022-02-28 23:56:41,2022-03-01 00:04:57,NaN,2.62,NaN,None,141,226,0,12.53,0.00,0.5,1.71,0.00,0.3,17.54,NaN,NaN


In [5]:
df_jan = pd.read_parquet('yellow_tripdata_2022-01.parquet')

In [6]:
df_jan['period'] = 1
df_feb['period'] = 2

In [7]:
df = pd.concat([df_jan, df_feb])

In [8]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,period
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,1
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,1
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,1
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,1
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979426,2,2022-02-28 23:50:00,2022-03-01 00:06:00,NaN,3.40,NaN,None,163,193,0,14.37,0.0,0.5,0.00,0.0,0.3,17.67,NaN,NaN,2
2979427,2,2022-02-28 23:06:57,2022-02-28 23:19:12,NaN,3.48,NaN,None,141,4,0,14.51,0.0,0.5,2.00,0.0,0.3,19.81,NaN,NaN,2
2979428,2,2022-02-28 23:48:13,2022-03-01 00:03:33,NaN,3.05,NaN,None,161,151,0,14.38,0.0,0.5,3.81,0.0,0.3,21.49,NaN,NaN,2
2979429,2,2022-02-28 23:56:41,2022-03-01 00:04:57,NaN,2.62,NaN,None,141,226,0,12.53,0.0,0.5,1.71,0.0,0.3,17.54,NaN,NaN,2


### Q2 - the standard deviation of the trips duration in January = 46 mins

In [9]:
def prep_timedelta(row):
    if (row['tpep_dropoff_datetime'] > row['tpep_pickup_datetime']):
        return (row['tpep_dropoff_datetime'] - row['tpep_pickup_datetime']).seconds 
    else:
        return 0


df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'])
df['duration_sec'] = df.apply(prep_timedelta, axis=1)
df['duration_min'] = df['duration_sec'] / 60

In [10]:
df.query('period==1').filter(like='duration').std()

duration        0 days 00:46:26.718308266
duration_sec                   2711.60561
duration_min                    45.193427
dtype: object

### Q3 - fraction of the records left after you dropped the outliers = 98%

In [12]:
df.query('duration_min >= 1 and duration_min <= 60').shape[0] / df.shape[0]

0.9809433949092491

### Q4 - the dimensionality of this matrix (number of columns)=515

In [13]:
df_sample = df.query('duration_min >= 1 and duration_min <= 60').copy()

In [14]:
df_sample['PULocation'] = df_sample['PULocationID'].astype('string')
df_sample['DOLocation'] = df_sample['DOLocationID'].astype('string')

In [16]:
len(list(set(df_sample.query('period==1')['PULocationID'].tolist())) + list(set(df_sample.query('period==1')['DOLocationID'].tolist())))

515

### Q5 

In [60]:
preprocessor = ColumnTransformer(
    transformers=[
        ('pickup_location', CountVectorizer(binary=True, vocabulary=list(set(df_sample.query('period==1')['PULocation'].tolist()))), 'PULocation'),
        ('dropoff_location', CountVectorizer(binary=True, vocabulary=list(set(df_sample.query('period==1')['DOLocation'].tolist()))), 'DOLocation'),
        ], remainder="drop")

In [61]:
preprocessor.fit_transform(df_sample.query('period==1')[['DOLocation', 'PULocation']])

<2421441x515 sparse matrix of type '<class 'numpy.int64'>'
	with 4814738 stored elements in Compressed Sparse Row format>

In [63]:
preprocessor.fit(df_sample.query('period==1')[['PULocation', 'DOLocation']])

ColumnTransformer(transformers=[('pickup_location',
                                 CountVectorizer(binary=True,
                                                 vocabulary=['89', '1', '189',
                                                             '193', '22', '261',
                                                             '133', '96', '254',
                                                             '164', '160',
                                                             '122', '218', '16',
                                                             '118', '130', '71',
                                                             '51', '53', '18',
                                                             '33', '62', '211',
                                                             '4', '173', '85',
                                                             '131', '5', '147',
                                                             '25', ...]),
                                 'PULocation'),
                                ('dropoff_location',
                                 CountVectorizer(binary=True,
                                                 vocabulary=['89', '1', '27',
                                                             '189', '84', '193',
                                                             '133', '261', '22',
                                                             '96', '254', '164',
                                                             '160', '122',
                                                             '218', '16', '118',
                                                             '130', '71', '51',
                                                             '53', '18', '33',
                                                             '62', '211', '4',
                                                             '173', '251', '85',
                                                             '131', ...]),
                                 'DOLocation')])

In [64]:
X_train = preprocessor.transform(df_sample.query('period==1')[['PULocation', 'DOLocation']])
X_test = preprocessor.transform(df_sample.query('period==2')[['PULocation', 'DOLocation']])
y_train = df_sample.query('period==1')['duration_min']
y_test = df_sample.query('period==2')['duration_min']

In [70]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

7.019665265319148

In [71]:
y_pred = lr.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

7.821571106346599